In [22]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
# model
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.9)

In [24]:
# parser
parser = StrOutputParser()

In [25]:
# create a state

class BlogState(TypedDict):

    topic : str
    outline : str
    content : str
    score : int

In [26]:
def create_outline(state : BlogState) -> BlogState:

      # fetch title
      topic = state['topic']

      # prompt template
      template = PromptTemplate.from_template(
            template = "generate the detailed outline on the topic : {topic} for generating a blog"
      )

      # chain
      chain = template | model | parser

      # ask the llm for the outline
      outline = chain.invoke({"topic" : topic})

      # insert the outline
      state['outline'] = outline

      return state

In [27]:
def gen_blog(state : BlogState) -> BlogState:

      # fetch title
      topic = state['topic']
      outline = state['outline']

      # prompt template
      template = PromptTemplate.from_template(
            template = "generate an detailed blog on the topic : {topic} by using the following outline : {outline}"
      )

      # chain
      chain = template | model | parser

      # ask the llm for the blog content
      content = chain.invoke({"outline" : outline, "topic" : topic})

      # insert the content
      state['content'] = content

      return state

In [28]:
def get_scores(state : BlogState) -> BlogState:
      
      # fetch title
      content = state['content']
      outline = state['outline']

      # prompt template
      template = PromptTemplate.from_template(
            template = "be the strict/genuine and experinced evaluater or experinced reader/consumer and based on this outline : {outline} rate my blog {content} from 1 to 10"
      )

      # chain
      chain = template | model | parser

      # ask the llm for the blog content
      score = chain.invoke({"outline" : outline, "content" : content})

      # insert the content
      state['score'] = score

      return state

In [30]:
# create our graph
graph = StateGraph(BlogState)

# add nodes
graph.add_node('create_outline', create_outline)
graph.add_node("gen_blog", gen_blog)
graph.add_node("get_scores", get_scores)

# add edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'gen_blog')
graph.add_edge('gen_blog', "get_scores")
graph.add_edge('get_scores', END)

# compile
workflow = graph.compile()

In [31]:
# execute

intial_state = {"topic" : "How and ways to write prompt"}

final_state = workflow.invoke(intial_state)

In [32]:
final_state

{'topic': 'How and ways to write prompt',
 'outline': '## Detailed Outline: How and Ways to Write Prompts for Generating a Blog\n\n### Introduction\n*   **A. The Rise of AI in Content Creation:** Briefly touch upon the capabilities of AI in generating text.\n*   **B. The Importance of Effective Prompt Writing:**\n    *   Garnering high-quality, relevant, and unique output.\n    *   Saving time and iteration cycles.\n    *   Ensuring alignment with desired tone, style, and goals.\n*   **C. What This Guide Covers:** A comprehensive framework for crafting impactful blog prompts.\n\n### I. Understanding the Fundamentals of Effective Prompt Writing (General Principles)\n*   **A. Clarity & Precision:**\n    *   Avoid ambiguity: Use direct and unambiguous language.\n    *   Define terms: If using niche jargon, briefly explain or instruct AI to understand context.\n*   **B. Specificity & Detail:**\n    *   The more detail, the better the output (within reason).\n    *   Think like a human edit

In [19]:
final_state['topic']

'How and ways to write prompt'

In [20]:
final_state['outline']

'This outline provides a comprehensive guide on how to effectively write prompts for AI tools to generate high-quality blog posts.\n\n---\n\n## Detailed Outline: How and Ways to Write Prompts for Generating a Blog Post\n\n**I. Introduction: The Power of Prompts in AI-Driven Content Creation**\n    A. The Rise of AI in Content Marketing\n        1. Efficiency and scalability\n        2. Overcoming writer\'s block\n        3. Ensuring consistency\n    B. What is a "Prompt" in this Context?\n        1. An instruction or query given to an AI model\n        2. The key to unlocking specific and relevant outputs\n    C. Why Effective Prompt Writing Matters for Blog Posts\n        1. Directly impacts the quality, relevance, and originality of generated content\n        2. Saves time on revisions and editing\n        3. Ensures alignment with content strategy and goals\n    D. Outline Overview: What this guide will cover\n\n**II. Foundations of Effective Prompt Writing: Core Principles**\n    A

In [21]:
final_state['content']

'## Beyond the Blank Page: Mastering the Art of AI Prompt Writing for Stellar Blog Posts\n\nThe digital content landscape is evolving at warp speed, and at the forefront of this revolution are Artificial Intelligence (AI) tools. From generating catchy headlines to drafting entire articles, AI is empowering content creators to achieve unprecedented levels of efficiency and scale. But here\'s the secret: the quality of the AI\'s output isn\'t magic; it\'s a direct reflection of the instructions it receives. This is where the art of "prompt writing" comes into play.\n\nIf you\'ve ever stared at a blank AI prompt box, wondering how to get more than just generic fluff, you\'re in the right place. This comprehensive guide will demystify the process, transforming you into a prompt engineer capable of coaxing high-quality, relevant, and original blog posts from your AI co-pilot.\n\n### I. Introduction: The Power of Prompts in AI-Driven Content Creation\n\n**A. The Rise of AI in Content Marketi

In [ ]:
final_state['scores']